# DerivKit — ForecastKit Fisher Bias Demo

This notebook demonstrates how to use `ForecastKit` to compute the **Fisher bias**
and the corresponding parameter shift $\Delta\theta$ caused by a systematic offset in the observables.

**Model (same as Fisher demo):**

- Observables:  
  $o_1 = \theta_1 + \theta_2$  
  $o_2 = \theta_1^2 + 2\,\theta_2^2$
- Jacobian:  
  $J(\theta_1, \theta_2) = \begin{bmatrix}1 & 1\\ 2\theta_1 & 4\theta_2\end{bmatrix}$

**What it does**
1. Defines the model and analytic Jacobian.
2. Sets fiducial parameters $\theta_0$ and identity data covariance $C$.
3. Creates a toy systematic offset $\Delta d$ in the observables so that  
   $\Delta\nu = d_{\text{with}} - d_{\text{without}} = \Delta d$.
4. Computes
   - Fisher matrix $F$ with `ForecastKit`,
   - Fisher bias vector and parameter shift $\Delta\theta$ with `ForecastKit`,
   - Analytic reference $\Delta\theta = F^{-1} J^\top C^{-1} \Delta\nu$.
5. Compares numeric vs analytic results.
6. Optionally plots **1σ** ellipses at $\theta_0$ and $\theta_0 + \Delta\theta$ and the bias arrow.

### Notes
- If `method` is omitted, the **adaptive** backend is used.
- First-order analytic bias formula (for small $\Delta\nu$):  
  $\Delta\theta \approx F^{-1} J^\top C^{-1} \Delta\nu$, where $F = J^\top C^{-1} J$ at $\theta_0$.


### Requirements
- ``derivkit`` importable in your environment.
- ``numpy`` for numerical computations.
- ``matplotlib`` for plotting.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from derivkit import ForecastKit

# 1D & 2D confidence levels
PCT_1D_1SIG = 0.682689492  # 68.27%
PCT_1D_2SIG = 0.954499736  # 95.45%
PCT_2D_1SIG = 0.683  # 68.3%
PCT_2D_2SIG = 0.955  # 95.5%

def plot_fisher_ellipse(theta0: np.ndarray,
                        fisher_matrix: np.ndarray,
                        *,
                        level: float = PCT_2D_1SIG,
                        ax=None,
                        label: str | None = None,
                        lw: float = 2.0,
                        ls: str = "-",
                        color: str | None = None):
    """Plots the Fisher ellipse for a 2×2 Fisher matrix `F2` at parameter point `theta0`.

    Args:
        theta0: Center of the ellipse (2D parameter point).
        fisher_matrix: Fisher matrix (shape 2x2).
        level: Confidence level for the ellipse (0.393, 0.683, 0.955, 0.997).
        ax: Matplotlib Axes to plot on (creates new if None).
        label: Label for the ellipse (for legend).
        lw: Line width for the ellipse.
        ls: Line style for the ellipse.
        color: Color of the ellipse.

    Returns:
        Matplotlib Axes with the ellipse plotted.
    """
    f_mat = np.asarray(fisher_matrix, dtype=float)
    if f_mat.shape != (2, 2):
        raise ValueError(f"Expected a 2×2 Fisher; got shape {f_mat.shape}.")
    if not np.allclose(f_mat, f_mat.T, rtol=1e-10, atol=1e-12):
        f_mat = 0.5 * (f_mat + f_mat.T)  # symmetrize

    # 2 dof chi2 quantiles mapping
    level_to_k2 = {0.393: 1.00, 0.683: 2.30, 0.955: 6.17, 0.997: 11.8}
    k2 = level_to_k2.get(level, 2.30)

    param_cov = np.linalg.pinv(f_mat, rcond=1e-12)
    vals, vecs = np.linalg.eigh(param_cov)  # ascending eigenvalues

    t = np.linspace(0, 2*np.pi, 400)
    circle = np.vstack([np.cos(t), np.sin(t)])
    ellipse = vecs @ np.diag(np.sqrt(vals * k2)) @ circle

    if not color:
        color= "#3b9ab2"
    if ax is None:
        _, ax = plt.subplots()
    ax.plot(theta0[0] + ellipse[0], theta0[1] + ellipse[1],
            label=label, linewidth=lw, linestyle=ls, color=color)
    ax.scatter([theta0[0]], [theta0[1]], s=25, color=color)
    ax.set_xlabel(r"parameter $\theta_1$", fontsize=13)
    ax.set_ylabel(r"parameter $\theta_2$", fontsize=13)
    ax.set_aspect("equal", adjustable="box")
    if label:
        ax.legend(frameon=False, fontsize=13)
    return ax


def model(param_list) -> np.ndarray:
    """Simple model: maps parameters(θ1, θ2) to observables (o1, o2) .

    o1 = θ1 + θ2
    o2 = θ1^2 + 2θ2^2

    Args:
        param_list: List or array of parameters [θ1, θ2].

    Returns:
        Array of observables [o1, o2].
    """
    theta1 = float(param_list[0])
    theta2 = float(param_list[1])
    obs1 = theta1 + theta2
    obs2 = theta1 * theta1 + 2.0 * theta2 * theta2
    return np.array([obs1, obs2], dtype=float)


def jacobian_analytic(theta1: float, theta2: float) -> np.ndarray:
    """Analytic Jacobian of `model`.

    Args:
        theta1: Parameter theta1.
        theta2: Parameter theta2.

    Returns:
        Jacobian matrix J (shape 2x2).
    """
    return np.array([[1.0, 1.0],
                     [2.0 * theta1, 4.0 * theta2]], dtype=float)


def fisher_analytic(theta1: float, theta2: float, cov: np.ndarray) -> np.ndarray:
    """Analytic Fisher matrix for the model at (x, y) with data covariance `cov`.

    Args:
        theta1: Parameter theta1.
        theta2: Parameter theta2.
        cov: Data covariance matrix (shape 2x2).

    Returns:
        Fisher information matrix (shape 2x2).
    """
    jac = jacobian_analytic(theta1, theta2)
    cov_inv = np.linalg.inv(cov)
    return jac.T @ cov_inv @ jac


def array_delta(name: str, num: np.ndarray, ref: np.ndarray) -> None:
    """Computes and prints the difference between two arrays.

    Args:
        name: Name of the array (for printing).
        num: Numeric array.
        ref: Reference (analytic) array.

    Returns:
        None
    """
    d = num - ref
    print(f"\n{name} (numeric):\n{num}")
    print(f"\n{name} (analytic):\n{ref}")
    print(f"\n $\\Delta$ = numeric - analytic:\n{d}")
    print(f"$\\max |\\Delta|$ = {np.max(np.abs(d)):.3e}, $|| \\Delta||_2$  = {np.linalg.norm(d):.3e}")


def analytic_bias_delta_theta(theta0: np.ndarray, cov: np.ndarray, delta_nu: np.ndarray) -> np.ndarray:
    """First-order analytic delta_theta from Fisher bias formula.

    Args:
        theta0: Fiducial parameter point (array of shape 2).
        cov: Data covariance matrix (shape 2x2).
        delta_nu: Systematic offset in observables (shape 2).

    Returns:
        Analytic delta_theta (shape 2)
    """
    jac = jacobian_analytic(theta0[0], theta0[1])
    cov_inv = np.linalg.inv(cov)
    fish = jac.T @ cov_inv @ jac
    return np.linalg.pinv(fish, rcond=1e-12) @ (jac.T @ cov_inv @ delta_nu)

print("Imports and helpers ready. A spark of joy!")


## Configure & Run
Set the backend `method`, the pseudo-inverse `rcond`, and the systematic offset `Δd`.
Run the cell to compute Fisher, Fisher bias, and the parameter shift. Toggle plotting as desired.

In [ ]:
# --- User-configurable knobs ---
method = "adaptive"  # "adaptive" or "finite"
rcond = 1e-12
delta_d = np.array([0.5, -0.2], dtype=float)  # systematic offset applied to observables
SHOW_PLOT = True

# Fiducial and covariance
theta0 = np.array([1.0, 2.0], dtype=float)
cov = np.eye(2)  # identity covariance matrix

print("=== Fisher Bias demo ===")
print("fiducial θ0 =", theta0)
print("covariance C =\n", cov)
print("backend method =", method)

# Observables with/without systematics
d_without = model(theta0)  # data vector without systematics
d_with = d_without + delta_d  # data vector with systematics
delta_nu = d_with - d_without  # equals delta_d here
print("model($\\theta_0$) =", d_without)
print("systematic $\\Delta d$ =", delta_d)
print("$\\Delta \\nu$ =", delta_nu)

# ForecastKit pipeline
fk = ForecastKit(function=model, theta0=theta0, cov=cov)
f_num = fk.fisher(method=method)
f_ref = fisher_analytic(theta0[0], theta0[1], cov)
array_delta("Fisher", f_num, f_ref)

bias_vec, dtheta_num = fk.fisher_bias(
    fisher_matrix=f_num,
    delta_nu=delta_nu,
    method=method,
    rcond=rcond,
)
dtheta_ref = analytic_bias_delta_theta(theta0, cov, delta_nu)
array_delta("$\\Delta \\theta$ (parameter shift)", dtheta_num, dtheta_ref)

# Report marginal σ and Δθ in σ-units
f_inv = np.linalg.pinv(f_num, rcond=rcond)
sigma = np.sqrt(np.diag(f_inv))
z_units = dtheta_num / sigma
print("\nParameter covariance (F^{-1}):\n", f_inv)
print("marginal $1 \\sigma$:", sigma)
print("$\\Delta \\theta$ in $\\sigma$-units:", z_units)

if SHOW_PLOT:
    fig, ax = plt.subplots()
    blue = "#3b9ab2"
    red = "#f21901"

    theta_biased = theta0 + dtheta_num

    # Plot 1σ at θ0 and at θ0+Δθ using the SAME Fisher (shapes identical)
    plot_fisher_ellipse(theta0, f_num, level=PCT_2D_1SIG,
                        ax=ax, label="fiducial $1 \\sigma$", ls="-", lw=2.0, color=red)
    plot_fisher_ellipse(theta_biased, f_num, level=PCT_2D_1SIG,
                        ax=ax, label="biased $1 \\sigma$", ls="-", lw=2.0, color=blue)
    ax.set_aspect("equal", adjustable="box")

    # Bias arrow θ0 → θ0 + Δθ
    ax.annotate("", xy=theta_biased, xytext=theta0,
                arrowprops=dict(arrowstyle="->", lw=2.0, color=blue))
    ax.scatter(*theta_biased, s=25, color=blue)
    ax.set_title("Fisher bias: $1 \\sigma$ contours (fiducial vs biased)", fontsize=15)
    plt.tight_layout()
    plt.show()

print("Done. Small steps in θ, big steps for understanding.")


### Tips
- To try finite differences, set `method = "finite"` above.
- Adjust `delta_d` to explore different systematic offsets.
- The two **1σ** ellipses use the *same* Fisher matrix and thus have identical sizes; only the centers differ.